In [57]:
#💡 If this notebook helped you, please upvote ⭐ and fork 🔁  
#It motivates me to release more practical ML & Pandas tutorials!

# 📘 Top 50 Pandas Tricks for ML Engineers  
### Fast feature engineering with `vectorized ops`, `groupby`, `assign`, `eval`, and `query`

In this notebook you’ll see **50 battle-tested Pandas tricks** that are especially useful for:

- **Feature engineering** for ML models  
- **Speeding up data pipelines** via vectorization  
- Writing **clean, chainable code** using `assign`, `eval`, and `query`  
- Doing powerful **aggregations** with `groupby`

We’ll use a **synthetic click-stream marketing dataset** that mimics real ML preprocessing flows.

---

## Table of Contents

1. [Setup & Synthetic Dataset](#setup)
2. [Vectorized Operations (Tricks 1–20)](#vectorized)
3. [Groupby Power Moves (Tricks 21–35)](#groupby)
4. [Feature Engineering with `assign` (Tricks 36–42)](#assign)
5. [Fast Expressions with `eval` (Tricks 43–46)](#eval)
6. [Readable Filtering with `query` (Tricks 47–50)](#query)

In [2]:
# ================================================================
# 1. Setup & Synthetic Dataset
# ================================================================
import numpy as np
import pandas as pd

pd.set_option("display.max_columns", 50)
pd.set_option("display.width", 120)

np.random.seed(42)

n = 1000

df = pd.DataFrame({
    "user_id": np.random.randint(1, 201, size=n),          # 200 users
    "session_id": np.random.randint(1, 5001, size=n),      # 5000 sessions
    "age": np.random.randint(18, 61, size=n),
    "country": np.random.choice(["US", "FR", "IN", "DE", "BR"], size=n),
    "device": np.random.choice(["mobile", "desktop", "tablet"], size=n),
    "clicks": np.random.poisson(lam=3, size=n),
    "impressions": np.random.poisson(lam=10, size=n) + 1,  # avoid zero
    "spent": np.round(np.random.exponential(scale=2, size=n), 2),
    "label": np.random.choice([0, 1], size=n, p=[0.7, 0.3])
})

# Add a datetime column
df["timestamp"] = pd.Timestamp("2024-01-01") + pd.to_timedelta(
    np.random.randint(0, 60*24*30, size=n), unit="m"
)

df.head()

,user_id,session_id,age,country,device,clicks,impressions,spent,label,timestamp
0,103,4008,39,DE,tablet,1,10,0.20,0,2024-01-10 16:24:00
1,180,2473,51,US,tablet,3,7,0.38,1,2024-01-25 15:45:00
2,93,2661,18,IN,desktop,4,12,0.22,1,2024-01-17 05:00:00
3,15,2740,43,DE,tablet,2,16,1.90,0,2024-01-21 20:25:00
4,107,1762,60,FR,mobile,2,13,0.20,1,2024-01-02 02:47:00


<a id="vectorized"></a>
## 2. Vectorized Operations (Tricks 1–20)

Vectorization = using **array-wise operations** instead of Python loops.  
This is the single biggest speed-up hack for ML preprocessing.

In [4]:
# Trick 1 — Vectorized CTR (click-through rate)
df["ctr"] = df["clicks"] / df["impressions"]
df[["clicks", "impressions", "ctr"]].head()

,clicks,impressions,ctr
0,1,10,0.100000
1,3,7,0.428571
2,4,12,0.333333
3,2,16,0.125000
4,2,13,0.153846


In [5]:
# Trick 2 — Avoid division by zero with vectorized where
df["ctr_safe"] = df["clicks"] / df["impressions"].where(df["impressions"] != 0, 1)
df[["clicks", "impressions", "ctr_safe"]].head()

,clicks,impressions,ctr_safe
0,1,10,0.100000
1,3,7,0.428571
2,4,12,0.333333
3,2,16,0.125000
4,2,13,0.153846


In [6]:
# Trick 3 — Vectorized log transform for skewed features
# (Tip: add 1 to avoid log(0))
df["log_spent"] = np.log1p(df["spent"])
df[["spent", "log_spent"]].head()

,spent,log_spent
0,0.20,0.182322
1,0.38,0.322083
2,0.22,0.198851
3,1.90,1.064711
4,0.20,0.182322


In [7]:
# Trick 4 — Clip outliers with vectorized clip (winsorization-lite)
df["spent_clipped"] = df["spent"].clip(lower=0, upper=10)
df[["spent", "spent_clipped"]].head()

,spent,spent_clipped
0,0.20,0.20
1,0.38,0.38
2,0.22,0.22
3,1.90,1.90
4,0.20,0.20


In [8]:
# Trick 5 — Standardize feature (z-score) in one line
spent_mean = df["spent"].mean()
spent_std = df["spent"].std()
df["spent_z"] = (df["spent"] - spent_mean) / spent_std
df[["spent", "spent_z"]].head()

,spent,spent_z
0,0.20,-0.941013
1,0.38,-0.847604
2,0.22,-0.930634
3,1.90,-0.058817
4,0.20,-0.941013


In [9]:
# Trick 6 — Bin continuous variable (age → buckets) with cut
age_bins = [18, 25, 35, 45, 60, np.inf]
age_labels = ["18-24", "25-34", "35-44", "45-60", "60+"]
df["age_bucket"] = pd.cut(df["age"], bins=age_bins, labels=age_labels, right=False)
df[["age", "age_bucket"]].head()

,age,age_bucket
0,39,35-44
1,51,45-60
2,18,18-24
3,43,35-44
4,60,60+


In [10]:
# Trick 7 — Fast boolean conditions with vectorized & and |
high_ctr_mask = (df["ctr"] > 0.3) & (df["impressions"] > 5)
df["high_engagement"] = high_ctr_mask.astype(int)
df[["ctr", "impressions", "high_engagement"]].head()

,ctr,impressions,high_engagement
0,0.100000,10,0
1,0.428571,7,1
2,0.333333,12,1
3,0.125000,16,0
4,0.153846,13,0


In [11]:
# Trick 8 — Vectorized multi-condition labeling with np.select
conditions = [
    df["ctr"] < 0.1,
    df["ctr"].between(0.1, 0.3),
    df["ctr"] > 0.3
]
choices = ["low", "medium", "high"]
df["ctr_segment"] = np.select(conditions, choices, default="unknown")
df[["ctr", "ctr_segment"]].head()

,ctr,ctr_segment
0,0.100000,medium
1,0.428571,high
2,0.333333,high
3,0.125000,medium
4,0.153846,medium


In [12]:
# Trick 9 — Fast missing value handling with fillna and isna
# (Introduce some NaNs for demo)
df_missing = df.copy()
df_missing.loc[df_missing.sample(frac=0.05, random_state=0).index, "spent"] = np.nan

df_missing["spent_filled"] = df_missing["spent"].fillna(df_missing["spent"].median())
df_missing[["spent", "spent_filled"]].head(10)

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,spent,spent_filled
0,0.20,0.20
1,NaN,1.44
2,0.22,0.22
3,1.90,1.90
4,0.20,0.20
5,2.81,2.81
6,0.04,0.04
7,0.57,0.57
8,4.43,4.43
9,2.54,2.54


In [13]:
# Trick 10 — Vectorized string operations (lower, contains)
df["country_lower"] = df["country"].str.lower()
df["is_eu"] = df["country"].isin(["FR", "DE"]).astype(int)
df[["country", "country_lower", "is_eu"]].head()

,country,country_lower,is_eu
0,DE,de,1
1,US,us,0
2,IN,in,0
3,DE,de,1
4,FR,fr,1


In [14]:
# Trick 11 — Extract datetime features in one shot
df["hour"] = df["timestamp"].dt.hour
df["dayofweek"] = df["timestamp"].dt.dayofweek
df["date"] = df["timestamp"].dt.date
df[["timestamp", "hour", "dayofweek", "date"]].head()

,timestamp,hour,dayofweek,date
0,2024-01-10 16:24:00,16,2,2024-01-10
1,2024-01-25 15:45:00,15,3,2024-01-25
2,2024-01-17 05:00:00,5,2,2024-01-17
3,2024-01-21 20:25:00,20,6,2024-01-21
4,2024-01-02 02:47:00,2,1,2024-01-02


In [15]:
# Trick 12 — Vectorized ranking for feature engineering
# Rank users by total spend (later we'll do by group properly)
df["spent_rank_global"] = df["spent"].rank(method="dense", ascending=False)
df[["spent", "spent_rank_global"]].head()

,spent,spent_rank_global
0,0.20,415.0
1,0.38,399.0
2,0.22,413.0
3,1.90,256.0
4,0.20,415.0


In [16]:
# Trick 13 — Replace values with map (e.g., encoding categories)
country_to_region = {
    "US": "NA",
    "FR": "EU",
    "DE": "EU",
    "IN": "APAC",
    "BR": "LATAM"
}
df["region"] = df["country"].map(country_to_region)
df[["country", "region"]].head()

,country,region
0,DE,EU
1,US,NA
2,IN,APAC
3,DE,EU
4,FR,EU


In [17]:
# Trick 14 — Vectorized cumulative sums per sorted time (global)
df_sorted = df.sort_values(["user_id", "timestamp"])
df_sorted["cum_clicks_global"] = df_sorted["clicks"].cumsum()
df_sorted[["user_id", "timestamp", "clicks", "cum_clicks_global"]].head()

,user_id,timestamp,clicks,cum_clicks_global
709,1,2024-01-08 10:51:00,6,6
927,1,2024-01-17 00:51:00,5,11
741,1,2024-01-19 06:29:00,4,15
137,1,2024-01-19 21:05:00,1,16
950,1,2024-01-25 13:31:00,1,17


In [18]:
# Trick 15 — Vectorized boolean to int & simple counts
df["is_desktop"] = (df["device"] == "desktop").astype(int)
df["is_mobile"] = (df["device"] == "mobile").astype(int)
df[["device", "is_desktop", "is_mobile"]].head()

,device,is_desktop,is_mobile
0,tablet,0,0
1,tablet,0,0
2,desktop,1,0
3,tablet,0,0
4,mobile,0,1


In [19]:
# Trick 16 — Combine multiple numeric features quickly
# Example: interaction feature "clicks * spent"
df["click_value"] = df["clicks"] * df["spent"]
df[["clicks", "spent", "click_value"]].head()

,clicks,spent,click_value
0,1,0.20,0.20
1,3,0.38,1.14
2,4,0.22,0.88
3,2,1.90,3.80
4,2,0.20,0.40


In [20]:
# Trick 17 — Vectorized quantile-based binning with qcut
df["spent_quantile"] = pd.qcut(df["spent"], q=4, labels=["Q1", "Q2", "Q3", "Q4"])
df[["spent", "spent_quantile"]].head()

,spent,spent_quantile
0,0.20,Q1
1,0.38,Q1
2,0.22,Q1
3,1.90,Q3
4,0.20,Q1


In [21]:
# Trick 18 — Use np.where for fast if-else style feature
df["heavy_user"] = np.where(df["clicks"] >= 5, 1, 0)
df[["clicks", "heavy_user"]].head()

,clicks,heavy_user
0,1,0
1,3,0
2,4,0
3,2,0
4,2,0


In [22]:
# Trick 19 — Efficient type conversion (downcasting) for memory
df["clicks_int16"] = pd.to_numeric(df["clicks"], downcast="integer")
df["spent_float32"] = pd.to_numeric(df["spent"], downcast="float")
df[["clicks", "clicks_int16", "spent", "spent_float32"]].head()

,clicks,clicks_int16,spent,spent_float32
0,1,1,0.20,0.20
1,3,3,0.38,0.38
2,4,4,0.22,0.22
3,2,2,1.90,1.90
4,2,2,0.20,0.20


In [23]:
# Trick 20 — Vectorized n-unique per row using axis=1 + nunique
# Example: distinct categories per user-session pair (country + device as toy)
df["nunique_cat_row"] = df[["country", "device"]].nunique(axis=1)
df[["country", "device", "nunique_cat_row"]].head()

,country,device,nunique_cat_row
0,DE,tablet,2
1,US,tablet,2
2,IN,desktop,2
3,DE,tablet,2
4,FR,mobile,2


<a id="groupby"></a>
## 3. Groupby Power Moves (Tricks 21–35)

`groupby` = instant **per-user, per-country, per-day** aggregations.  
This is where a lot of **ML feature engineering** happens.

In [24]:
# Trick 21 — Basic groupby aggregation (user-level features)
user_agg = (
    df.groupby("user_id")
      .agg(
          sessions=("session_id", "nunique"),
          total_clicks=("clicks", "sum"),
          total_impressions=("impressions", "sum"),
          total_spent=("spent", "sum")
      )
      .reset_index()
)
user_agg.head()

,user_id,sessions,total_clicks,total_impressions,total_spent
0,1,8,25,89,11.53
1,2,5,13,58,9.44
2,3,6,17,65,16.43
3,4,4,14,37,9.02
4,5,6,15,71,12.99


In [25]:
# Trick 22 — Create derived feature from groupby result (user CTR)
user_agg["user_ctr"] = user_agg["total_clicks"] / user_agg["total_impressions"]
user_agg.head()

,user_id,sessions,total_clicks,total_impressions,total_spent,user_ctr
0,1,8,25,89,11.53,0.280899
1,2,5,13,58,9.44,0.224138
2,3,6,17,65,16.43,0.261538
3,4,4,14,37,9.02,0.378378
4,5,6,15,71,12.99,0.211268


In [26]:
# Trick 23 — Multi-key groupby (user + device)
user_device_agg = (
    df.groupby(["user_id", "device"])
      .agg(
          clicks=("clicks", "sum"),
          impressions=("impressions", "sum")
      )
      .reset_index()
)
user_device_agg.head()

,user_id,device,clicks,impressions
0,1,desktop,4,13
1,1,mobile,7,39
2,1,tablet,14,37
3,2,desktop,10,34
4,2,mobile,2,9


In [27]:
# Trick 24 — Named aggregation syntax for multiple stats
country_stats = (
    df.groupby("country")
      .agg(
          mean_ctr=("ctr", "mean"),
          median_ctr=("ctr", "median"),
          sessions=("session_id", "nunique"),
          users=("user_id", "nunique")
      )
      .reset_index()
)
country_stats

,country,mean_ctr,median_ctr,sessions,users
0,BR,0.289882,0.272727,207,132
1,DE,0.291843,0.272727,202,128
2,FR,0.289208,0.266667,164,112
3,IN,0.300583,0.266667,204,137
4,US,0.285250,0.250000,211,134


In [28]:
# Trick 25 — Using transform to broadcast group-level stats back to rows
# Example: user-level mean CTR per row
df["user_mean_ctr"] = df.groupby("user_id")["ctr"].transform("mean")
df[["user_id", "ctr", "user_mean_ctr"]].head()

,user_id,ctr,user_mean_ctr
0,103,0.100000,0.211600
1,180,0.428571,0.278415
2,93,0.333333,0.267936
3,15,0.125000,0.208166
4,107,0.153846,0.297494


In [29]:
# Trick 26 — Groupby + rank within group
# Rank each user's sessions by timestamp
df_sorted = df.sort_values(["user_id", "timestamp"])
df_sorted["session_rank"] = df_sorted.groupby("user_id")["timestamp"].rank(method="first")
df_sorted[["user_id", "timestamp", "session_rank"]].head()

,user_id,timestamp,session_rank
709,1,2024-01-08 10:51:00,1.0
927,1,2024-01-17 00:51:00,2.0
741,1,2024-01-19 06:29:00,3.0
137,1,2024-01-19 21:05:00,4.0
950,1,2024-01-25 13:31:00,5.0


In [30]:
# Trick 27 — Cumulative sum per group (user-level cumulative clicks)
df_sorted["user_cum_clicks"] = df_sorted.groupby("user_id")["clicks"].cumsum()
df_sorted[["user_id", "timestamp", "clicks", "user_cum_clicks"]].head()

,user_id,timestamp,clicks,user_cum_clicks
709,1,2024-01-08 10:51:00,6,6
927,1,2024-01-17 00:51:00,5,11
741,1,2024-01-19 06:29:00,4,15
137,1,2024-01-19 21:05:00,1,16
950,1,2024-01-25 13:31:00,1,17


In [31]:
# Trick 28 — Rolling window features within group
# 3-session rolling mean CTR per user
df_sorted["user_ctr_rolling3"] = (
    df_sorted
    .groupby("user_id")["ctr"]
    .rolling(window=3, min_periods=1)
    .mean()
    .reset_index(level=0, drop=True)
)
df_sorted[["user_id", "timestamp", "ctr", "user_ctr_rolling3"]].head(10)

,user_id,timestamp,ctr,user_ctr_rolling3
709,1,2024-01-08 10:51:00,0.545455,0.545455
927,1,2024-01-17 00:51:00,0.357143,0.451299
741,1,2024-01-19 06:29:00,0.307692,0.403430
137,1,2024-01-19 21:05:00,0.166667,0.277167
950,1,2024-01-25 13:31:00,0.111111,0.195157
490,1,2024-01-26 07:11:00,0.250000,0.175926
536,1,2024-01-28 15:45:00,0.083333,0.148148
825,1,2024-01-30 13:13:00,0.333333,0.222222
218,2,2024-01-01 18:50:00,0.545455,0.545455
281,2,2024-01-04 22:16:00,0.000000,0.272727


In [32]:
# Trick 29 — Groupby + size for quick frequency encoding
country_freq = df["country"].value_counts(normalize=True).rename("country_freq")
df = df.join(country_freq, on="country")
df[["country", "country_freq"]].head()

,country,country_freq
0,DE,0.205
1,US,0.212
2,IN,0.207
3,DE,0.205
4,FR,0.167


In [33]:
# Trick 30 — Aggregating into list for sequence features
user_sessions_seq = (
    df.sort_values(["user_id", "timestamp"])
      .groupby("user_id")["device"]
      .agg(list)
      .reset_index(name="device_sequence")
)
user_sessions_seq.head()

,user_id,device_sequence
0,1,"[tablet, tablet, desktop, mobile, mobile, tabl..."
1,2,"[desktop, desktop, desktop, tablet, mobile]"
2,3,"[tablet, desktop, mobile, tablet, desktop, mob..."
3,4,"[desktop, tablet, tablet, desktop]"
4,5,"[mobile, mobile, desktop, mobile, mobile, mobile]"


In [34]:
# Trick 31 — Pivot table for wide-format features
country_device_pivot = (
    df.pivot_table(
        index="country",
        columns="device",
        values="clicks",
        aggfunc="mean",
        fill_value=0
    )
)
country_device_pivot

device,desktop,mobile,tablet
country,,,
BR,3.083333,3.028571,2.671642
DE,2.984615,2.791667,2.955882
FR,3.029851,3.080000,2.860000
IN,3.318182,2.954545,2.880000
US,3.084746,2.514706,3.023529


In [35]:
# Trick 32 — Groupby on time-based feature (per-day stats)
df["date"] = pd.to_datetime(df["timestamp"].dt.date)
daily_stats = (
    df.groupby("date")
      .agg(
          total_clicks=("clicks", "sum"),
          total_impressions=("impressions", "sum"),
          total_spent=("spent", "sum")
      )
)
daily_stats.head()

,total_clicks,total_impressions,total_spent
date,,,
2024-01-01,95,303,58.35
2024-01-02,81,318,62.43
2024-01-03,83,306,49.81
2024-01-04,67,277,68.73
2024-01-05,94,399,51.38


In [36]:
# Trick 33 — Groupby with multiple aggregations on same column
age_country_stats = (
    df.groupby(["age_bucket", "country"])["ctr"]
      .agg(["mean", "median", "std", "count"])
      .reset_index()
)
age_country_stats.head()

/tmp/ipykernel_47/4082623244.py:3: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby(["age_bucket", "country"])["ctr"]


,age_bucket,country,mean,median,std,count
0,18-24,BR,0.300721,0.250000,0.199837,42
1,18-24,DE,0.249822,0.200000,0.140532,35
2,18-24,FR,0.315996,0.250000,0.206645,25
3,18-24,IN,0.223702,0.190909,0.135681,32
4,18-24,US,0.259331,0.222222,0.160097,31


In [37]:
# Trick 34 — Using groupby + ngroup for group IDs (integer encoding groups)
df["country_group_id"] = df.groupby("country").ngroup()
df[["country", "country_group_id"]].head()

,country,country_group_id
0,DE,1
1,US,4
2,IN,3
3,DE,1
4,FR,2


In [38]:
# Trick 35 — Groupby, then filtering groups based on condition
# Example: keep only users with at least 5 sessions
user_session_counts = df.groupby("user_id")["session_id"].nunique()
valid_users = user_session_counts[user_session_counts >= 5].index
df_filtered_users = df[df["user_id"].isin(valid_users)]
df_filtered_users["user_id"].nunique(), df["user_id"].nunique()

(110, 198)

<a id="assign"></a>
## 4. Feature Engineering with `assign` (Tricks 36–42)

`assign` lets you **add multiple columns in a single chain**.  
This makes your ML preprocessing steps more readable and composable.

In [39]:
# Trick 36 — Basic assign for chained feature engineering
df_assign_demo = (
    df
    .assign(
        ctr=lambda d: d["clicks"] / d["impressions"],
        log_spent=lambda d: np.log1p(d["spent"]),
        click_value=lambda d: d["clicks"] * d["spent"]
    )
)
df_assign_demo.head()

,user_id,session_id,age,country,device,clicks,impressions,spent,label,timestamp,ctr,ctr_safe,log_spent,spent_clipped,spent_z,age_bucket,high_engagement,ctr_segment,country_lower,is_eu,hour,dayofweek,date,spent_rank_global,region,is_desktop,is_mobile,click_value,spent_quantile,heavy_user,clicks_int16,spent_float32,nunique_cat_row,user_mean_ctr,country_freq,country_group_id
0,103,4008,39,DE,tablet,1,10,0.20,0,2024-01-10 16:24:00,0.100000,0.100000,0.182322,0.20,-0.941013,35-44,0,medium,de,1,16,2,2024-01-10,415.0,EU,0,0,0.20,Q1,0,1,0.20,2,0.211600,0.205,1
1,180,2473,51,US,tablet,3,7,0.38,1,2024-01-25 15:45:00,0.428571,0.428571,0.322083,0.38,-0.847604,45-60,1,high,us,0,15,3,2024-01-25,399.0,NA,0,0,1.14,Q1,0,3,0.38,2,0.278415,0.212,4
2,93,2661,18,IN,desktop,4,12,0.22,1,2024-01-17 05:00:00,0.333333,0.333333,0.198851,0.22,-0.930634,18-24,1,high,in,0,5,2,2024-01-17,413.0,APAC,1,0,0.88,Q1,0,4,0.22,2,0.267936,0.207,3
3,15,2740,43,DE,tablet,2,16,1.90,0,2024-01-21 20:25:00,0.125000,0.125000,1.064711,1.90,-0.058817,35-44,0,medium,de,1,20,6,2024-01-21,256.0,EU,0,0,3.80,Q3,0,2,1.90,2,0.208166,0.205,1
4,107,1762,60,FR,mobile,2,13,0.20,1,2024-01-02 02:47:00,0.153846,0.153846,0.182322,0.20,-0.941013,60+,0,medium,fr,1,2,1,2024-01-02,415.0,EU,0,1,0.40,Q1,0,2,0.20,2,0.297494,0.167,2


In [40]:
# Trick 37 — assign + groupby.transform in one pipeline
df_user_features = (
    df
    .assign(
        ctr=lambda d: d["clicks"] / d["impressions"],
        user_mean_ctr=lambda d: d.groupby("user_id")["ctr"].transform("mean"),
        user_total_spent=lambda d: d.groupby("user_id")["spent"].transform("sum")
    )
)
df_user_features[["user_id", "ctr", "user_mean_ctr", "user_total_spent"]].head()

,user_id,ctr,user_mean_ctr,user_total_spent
0,103,0.100000,0.211600,14.36
1,180,0.428571,0.278415,9.27
2,93,0.333333,0.267936,14.62
3,15,0.125000,0.208166,15.76
4,107,0.153846,0.297494,10.44


In [41]:
# Trick 38 — assign with conditional logic
df_segmented = (
    df
    .assign(
        ctr=lambda d: d["clicks"] / d["impressions"],
        engagement_segment=lambda d: np.where(
            d["ctr"] > 0.3, "high",
            np.where(d["ctr"] > 0.15, "medium", "low")
        )
    )
)
df_segmented[["ctr", "engagement_segment"]].head()

,ctr,engagement_segment
0,0.100000,low
1,0.428571,high
2,0.333333,high
3,0.125000,low
4,0.153846,medium


In [42]:
# Trick 39 — assign + type conversion for memory optimization
df_memory_optimized = (
    df
    .assign(
        clicks=lambda d: pd.to_numeric(d["clicks"], downcast="integer"),
        impressions=lambda d: pd.to_numeric(d["impressions"], downcast="integer"),
        spent=lambda d: pd.to_numeric(d["spent"], downcast="float")
    )
)
df_memory_optimized.dtypes

user_id                       int64
session_id                    int64
age                           int64
country                      object
device                       object
clicks                         int8
impressions                    int8
spent                       float32
label                         int64
timestamp            datetime64[ns]
ctr                         float64
ctr_safe                    float64
log_spent                   float64
spent_clipped               float64
spent_z                     float64
age_bucket                 category
high_engagement               int64
ctr_segment                  object
country_lower                object
is_eu                         int64
hour                          int32
dayofweek                     int32
date                 datetime64[ns]
spent_rank_global           float64
region                       object
is_desktop                    int64
is_mobile                     int64
click_value                 

In [43]:
# Trick 40 — assign with intermediate columns reused later
df_chain = (
    df
    .assign(
        ctr=lambda d: d["clicks"] / d["impressions"],
        ctr_bucket=lambda d: pd.cut(
            d["ctr"], bins=[0, 0.1, 0.3, 1.0], labels=["low", "medium", "high"], include_lowest=True
        ),
        high_value_session=lambda d: (d["spent"] > 5) & (d["ctr"] > 0.2)
    )
)
df_chain[["clicks", "impressions", "ctr", "ctr_bucket", "high_value_session"]].head()

,clicks,impressions,ctr,ctr_bucket,high_value_session
0,1,10,0.100000,low,False
1,3,7,0.428571,high,False
2,4,12,0.333333,high,False
3,2,16,0.125000,medium,False
4,2,13,0.153846,medium,False


In [44]:
# Trick 41 — assign and pipe for very readable pipelines
def add_time_features(d):
    return d.assign(
        hour=d["timestamp"].dt.hour,
        dayofweek=d["timestamp"].dt.dayofweek
    )

df_time_features = df.pipe(add_time_features)
df_time_features[["timestamp", "hour", "dayofweek"]].head()

,timestamp,hour,dayofweek
0,2024-01-10 16:24:00,16,2
1,2024-01-25 15:45:00,15,3
2,2024-01-17 05:00:00,5,2
3,2024-01-21 20:25:00,20,6
4,2024-01-02 02:47:00,2,1


In [45]:
# Trick 42 — Chain everything: filter → assign → groupby → reset_index
user_country_agg_pipeline = (
    df
    .query("spent > 1")                             # filter
    .assign(ctr=lambda d: d["clicks"] / d["impressions"])
    .groupby(["user_id", "country"])
    .agg(
        mean_ctr=("ctr", "mean"),
        total_spent=("spent", "sum"),
        sessions=("session_id", "nunique")
    )
    .reset_index()
)
user_country_agg_pipeline.head()

,user_id,country,mean_ctr,total_spent,sessions
0,1,DE,0.083333,5.52,1
1,1,FR,0.303571,3.41,2
2,1,US,0.307692,1.63,1
3,2,BR,0.333333,4.15,1
4,2,FR,0.144444,2.36,2


<a id="eval"></a>
## 5. Fast Expressions with `eval` (Tricks 43–46)

`eval` lets you write **string expressions** that Pandas parses once and executes quickly.  
Useful for **vectorized math** and sometimes memory speed-ups on large data.

In [46]:
# Trick 43 — Basic eval for column expressions
df_eval = df.copy()
df_eval.eval(
    "ctr = clicks / impressions",
    inplace=True
)
df_eval[["clicks", "impressions", "ctr"]].head()

,clicks,impressions,ctr
0,1,10,0.100000
1,3,7,0.428571
2,4,12,0.333333
3,2,16,0.125000
4,2,13,0.153846


In [47]:
# Trick 44 — Multiple expressions with eval
df_eval = df_eval.eval(
    """
    click_value = clicks * spent
    cost_per_click = spent / clicks
    """,
)
df_eval[["clicks", "spent", "click_value", "cost_per_click"]].head()

,clicks,spent,click_value,cost_per_click
0,1,0.20,0.20,0.200000
1,3,0.38,1.14,0.126667
2,4,0.22,0.88,0.055000
3,2,1.90,3.80,0.950000
4,2,0.20,0.40,0.100000


In [51]:
# Trick 45 — eval with boolean logic, then cast outside eval if needed
df_eval = df.copy()

# 1) Use eval for the boolean expression
df_eval = df_eval.eval("heavy_user = clicks >= 5")

# 2) (Optional) convert True/False → 1/0 outside eval
df_eval["heavy_user"] = df_eval["heavy_user"].astype("int64")

df_eval[["clicks", "heavy_user"]].head()

,clicks,heavy_user
0,1,0
1,3,0
2,4,0
3,2,0
4,2,0


In [52]:
# Trick 46 — eval for chained expressions (single-expression per call)

df_eval_temp = df.copy()

# First expression: create ctr
df_eval_temp = df_eval_temp.eval(
    "ctr = clicks / impressions",
    engine="numexpr"
)

# Second expression: create log_ctr using ctr
df_eval_temp = df_eval_temp.eval(
    "log_ctr = log(ctr + 1e-6)",
    engine="numexpr"
)

df_eval_temp[["ctr", "log_ctr"]].head()

,ctr,log_ctr
0,0.100000,-2.302575
1,0.428571,-0.847296
2,0.333333,-1.098609
3,0.125000,-2.079434
4,0.153846,-1.871796


<a id="query"></a>
## 6. Readable Filtering with `query` (Tricks 47–50)

`query` gives you a **SQL-like syntax** for filtering rows.  
It’s great for complex conditions and readable ML data filters.

In [53]:
# Trick 47 — Basic query filtering
high_spend_sessions = df.query("spent > 5")
high_spend_sessions[["spent"]].head()

,spent
16,10.07
28,5.17
81,5.30
96,7.46
108,6.71


In [54]:
# Trick 48 — Complex conditions with query (AND / OR)
eu_high_ctr = df.query(
    "country in ['FR', 'DE'] and ctr > 0.3 and impressions > 5"
)
eu_high_ctr[["country", "ctr", "impressions"]].head()

,country,ctr,impressions
12,DE,0.375000,8
17,DE,0.444444,9
27,DE,0.400000,10
35,FR,0.357143,14
69,DE,0.571429,7


In [55]:
# Trick 49 — Using @variable inside query
min_age = 30
selected = df.query("age >= @min_age and device == 'mobile'")
selected[["age", "device"]].head()

,age,device
4,60,mobile
6,47,mobile
7,35,mobile
11,57,mobile
13,44,mobile


In [56]:
# Trick 50 — Chain query + assign + groupby for clean ML filters
ml_ready = (
    df
    .query("spent > 0 and impressions >= 5")      # filter bad rows
    .assign(
        ctr=lambda d: d["clicks"] / d["impressions"],
        log_spent=lambda d: np.log1p(d["spent"])
    )
    .query("ctr <= 1")                           # sanity filter
)

ml_features = (
    ml_ready
    .groupby(["user_id", "country"])
    .agg(
        mean_ctr=("ctr", "mean"),
        mean_log_spent=("log_spent", "mean"),
        sessions=("session_id", "nunique")
    )
    .reset_index()
)

ml_features.head()

,user_id,country,mean_ctr,mean_log_spent,sessions
0,1,DE,0.268308,0.557153,4
1,1,FR,0.303571,0.962956,2
2,1,US,0.237179,0.712204,2
3,2,BR,0.333333,1.638997,1
4,2,FR,0.144444,0.779062,2


---

## ✅ Summary

In this notebook you learned **50 Pandas tricks** optimized for ML pipelines:

- **Vectorized ops** for fast math and feature engineering  
- **Groupby** for powerful user-, country-, and time-level aggregates  
- **assign** for clean, chainable transformations  
- **eval** for fast vectorized expressions  
- **query** for readable filtering logic

You can now:

- Turn raw click-stream-like data into a rich feature table  
- Keep your Pandas code **fast**, **readable**, and **ML-ready**

If you fork this notebook on Kaggle, feel free to:
- Swap in a real dataset  
- Add model training (e.g., XGBoost / LightGBM) using `ml_features`  
- Benchmark `eval`/`query` vs. vanilla Pandas on larger data

Happy coding 🐼🚀